In [4]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as exu, estimator_util as esu, evaluation_util as evu, matcher_utils as mu
import importlib
from typing import List, Dict, Any
import wrappers as wrp
import os
import numpy as np

In [2]:
DATA_SRC_DIR = '/share/project_data/'
EXT_DATA_DIR = '/share/project_data/external/' # Data from kaggle
INT_DATA_DIR = '/share/project_data/internal' # Save mid progress

In [8]:
import cv2
import numpy as np

def calculate_covisibility(image1_path, image2_path):
    # Load the two images
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded.")

    # Initialize the ORB detector
    orb = cv2.ORB_create()

    # Detect keypoints and descriptors
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # Initialize the BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors
    matches = bf.match(des1, des2)

    # Sort matches by distance (quality of match)
    matches = sorted(matches, key=lambda x: x.distance)

    # Calculate covisibility as the ratio of matches to the average number of keypoints
    num_matches = len(matches)
    avg_keypoints = (len(kp1) + len(kp2)) / 2
    covisibility = num_matches / avg_keypoints if avg_keypoints > 0 else 0

    # Visualize matches (optional)
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    return covisibility, img_matches

# Example usage
image1_path = "//share/project_data/external/train/trevi_fountain/images/99678699_7739302088.jpg"
image2_path = "/share/project_data/external/train/trevi_fountain/images/66225128_7739308762.jpg"
covisibility, matched_image = calculate_covisibility(image1_path, image2_path)

print(f"Covisibility: {covisibility}")

Covisibility: 0.534


In [9]:
import cv2
import numpy as np

def calculate_covisibility_sift(image1_path, image2_path):
    # Load the two images
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded.")

    # Initialize the SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and descriptors
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Initialize the BFMatcher with L2 norm (used for SIFT)
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Match descriptors
    matches = bf.match(des1, des2)

    # Sort matches by distance (quality of match)
    matches = sorted(matches, key=lambda x: x.distance)

    # Calculate covisibility as the ratio of matches to the average number of keypoints
    num_matches = len(matches)
    avg_keypoints = (len(kp1) + len(kp2)) / 2
    covisibility = num_matches / avg_keypoints if avg_keypoints > 0 else 0

    # Visualize matches (optional)
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    return covisibility, img_matches

# Example usage
image1_path = "//share/project_data/external/train/trevi_fountain/images/99678699_7739302088.jpg"
image2_path = "/share/project_data/external/train/trevi_fountain/images/66225128_7739308762.jpg"
covisibility, matched_image = calculate_covisibility_sift(image1_path, image2_path)

print(f"Covisibility: {covisibility}")



Covisibility: 0.5941318112904664


: 

: 

: 

In [ ]:
import cv2
import numpy as np
import pandas as pd
import path

def calculate_covisibility_balanced_flow(image1_path, image2_path):
    # Load the two images as grayscale
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded.")

    # Resize images for consistent analysis
    img1 = cv2.resize(img1, (640, 480))
    img2 = cv2.resize(img2, (640, 480))

    # Calculate dense optical flow
    flow = cv2.calcOpticalFlowFarneback(
        img1, img2,
        None,  # Output flow array
        0.5,   # Pyramid scale
        3,     # Number of pyramid levels
        15,    # Window size
        3,     # Number of iterations at each pyramid level
        5,     # Size of the pixel neighborhood
        1.2,   # Standard deviation of the Gaussian
        0      # Flags
    )

    # Compute magnitude and angle of flow vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Adjust thresholds for valid motion
    magnitude_threshold_min = 1.5  # Looser minimum magnitude
    magnitude_threshold_max = 30.0  # Keep a tighter upper bound
    valid_flow_mask = (magnitude > magnitude_threshold_min) & (magnitude < magnitude_threshold_max)

    # Focus on regions with texture (high gradient regions)
    gradient_x = cv2.Sobel(img1, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(img1, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    texture_threshold = 20.0  # Looser texture threshold
    texture_mask = gradient_magnitude > texture_threshold

    # Combine valid flow mask with texture mask using weights
    combined_mask = valid_flow_mask & texture_mask
    combined_weighted_mask = 0.7 * valid_flow_mask + 0.3 * texture_mask

    # Normalize weights to scale between 0 and 1
    combined_weighted_mask = combined_weighted_mask / combined_weighted_mask.max()

    # Calculate covisibility as the weighted sum of valid motion and texture
    total_pixels = np.prod(flow.shape[:2])  # Total number of pixels
    valid_pixels = np.sum(combined_mask)
    weighted_pixels = np.sum(combined_weighted_mask)
    covisibility = weighted_pixels / total_pixels

    return covisibility, combined_mask


def process_csv_and_calculate_covisibility(csv_path, image_dir):
    # Read the CSV
    df = pd.read_csv(csv_path)

    # Add a new column to store covisibility results
    df['calc_covisibility'] = None 

    for index, row in df.iterrows():
        im1_path = f"{image_dir}/{row['im1']}.jpg"
        im2_path = f"{image_dir}/{row['im2']}.jpg"
        print(im1_path)
        if not os.path.exists(im1_path):
            print("missing")
        # Load the images
        img1 = cv2.imread(im1_path, cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(im2_path, cv2.IMREAD_GRAYSCALE)

        if img1 is None or img2 is None:
            print(f"Warning: One of the images {im1_path} or {im2_path} could not be loaded.")
            continue

        # Calculate covisibility
        covisibility = calculate_covisibility_balanced_flow(img1, img2)
        df.at[index, 'calc_covisibility'] = covisibility

        print(f"Processed: {im1_path} & {im2_path} -> Covisibility: {covisibility}")

    # Save the updated CSV with covisibility scores
    output_csv_path = "output_with_covisibility.csv"
    df.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")

csv_path = "/share/project_data/external/train/trevi_fountain/pair_covisibility.csv"  # Path to the input CSV file
image_dir = "/share/project_data/external/train/trevi_fountain/images"  # Directory containing the images
process_csv_and_calculate_covisibility(csv_path, image_dir)
"""
# Example usage
image1_path = "/share/project_data/external/train/trevi_fountain/images/76104794_5487278337.jpg"
image2_path = "/share/project_data/external/train/trevi_fountain/images/55093169_11139497355.jpg"
covisibility, matched_image = calculate_covisibility_balanced_flow(image1_path, image2_path)

print(f"Covisibility: {covisibility}")
"""


/share/project_data/external/train/trevi_fountain/images/99678699_7739302088.jpg


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object


In [5]:
importlib.reload(wrp)
importlib.reload(du)
importlib.reload(exu)
importlib.reload(esu)

<module 'utils.estimator_util' from '/app/utils/estimator_util.py'>

In [6]:
# Usage
importlib.reload(pu)
image_processor = pu.ImagePreprocessor(resize=(512, 512), normalize=False, greyscale=True)
importlib.reload(exu)
feature_extracotr = exu.FeatureExtractor(exu.ExtrectorType.SIFT)

In [7]:
importlib.reload(du)
dataset = du.DatasetLoader(root_dir=EXT_DATA_DIR, train_mode=True)
dataset.load_all_dataset(image_processor, feature_extracotr)

Loading image data and metadata
Extracting features from sences


Scenes: 100%|██████████| 12/12 [00:50<00:00,  4.21s/it]


In [1]:
from utils import matcher_utils as mu
importlib.reload(mu)
matcher = mu.FeatureMatcher(mu.MatcherType.BFMatcherWithKNN)

NameError: name 'importlib' is not defined

In [24]:
wrp.sample_pairs_for_run(dataset, 1000)
wrp.match_features(dataset, matcher, covisibility_threshold=0.1)

[+] Processing scene "brandenburg_gate": found 6767 pairs (will keep 1000)
[+] Processing scene "british_museum": found 889 pairs (will keep 889)
[+] Processing scene "buckingham_palace": found 6646 pairs (will keep 1000)
[+] Processing scene "colosseum_exterior": found 7314 pairs (will keep 1000)
[+] Processing scene "lincoln_memorial_statue": found 6308 pairs (will keep 1000)
[+] Processing scene "notre_dame_front_facade": found 8448 pairs (will keep 1000)
[+] Processing scene "pantheon_exterior": found 9888 pairs (will keep 1000)
[+] Processing scene "sacre_coeur": found 4233 pairs (will keep 1000)
[+] Processing scene "sagrada_familia": found 2767 pairs (will keep 1000)
[+] Processing scene "taj_mahal": found 6730 pairs (will keep 1000)
[+] Processing scene "temple_nara_japan": found 8115 pairs (will keep 1000)
[+] Processing scene "trevi_fountain": found 11166 pairs (will keep 1000)
Matching features for scene: brandenburg_gate
In matcher there are 1000 valid pairs to estimate for

In [19]:
list1 = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10), (11, 12), (13, 14), (15, 16)]
list2 = [(17, 18), (19, 20), (21, 22), (23, 24), (25, 26), (27, 28), (29, 30), (31, 32)]

In [20]:
importlib.reload(esu)

# Recreate the estimator configuration and estimator using the reloaded module
es_alg_conf = esu.RANSACConfig()
es_conf = esu.EstimatorConfig('RANSAC', es_alg_conf)
fe = esu.FundamentalMatrixEstimator(list1, list2, es_conf)


In [21]:
sub = wrp.estimate_fundamental_matrix(dataset=dataset, estimator=fe)

Estimating fundamental matrix for pairs in scene: brandenburg_gate
Estimating fundamental matrix for pairs in scene: british_museum
Estimating fundamental matrix for pairs in scene: buckingham_palace
Estimating fundamental matrix for pairs in scene: colosseum_exterior
Estimating fundamental matrix for pairs in scene: lincoln_memorial_statue
Estimating fundamental matrix for pairs in scene: notre_dame_front_facade
Estimating fundamental matrix for pairs in scene: pantheon_exterior
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
Estimating fundamental matrix for pairs in scene: sacre_coeur
Estimating fundamental matrix for pairs in scene: sagrada_familia
Estimating fundamental matrix for pairs in scene: taj_mahal
Estimating fundamental matrix f

In [22]:
sub

,sample_id,fundamental_matrix,mask,inliers1,inliers2
0,brandenburg_gate;90920828_5082887495-20133057_...,"[2.690539584220932, 1612.514617442752, 200.763...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.449611, -0.054365], [-0.4438, -0.063091],...","[[-0.466144, -0.055466], [-0.116896, -0.039746..."
1,brandenburg_gate;90920828_5082887495-17262282_...,"[128.78031623657645, 931.3318786717768, 291.45...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.449611, -0.054365], [-0.44568, -0.184222]...","[[-0.461805, -0.083045], [-0.021437, -0.182804..."
2,brandenburg_gate;20133057_3035445116-17262282_...,"[3.616088386231112, 40.25542356502047, 9.52606...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.47035, -0.043227], [-0.466144, -0.055466]...","[[-0.467542, -0.070519], [-0.461805, -0.083045..."
3,brandenburg_gate;38600512_2168650655-17262282_...,"[-6.738109899724764, 15.469393050221587, 3.294...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.473136, -0.038246], [-0.472237, -0.062819...","[[-0.091041, -0.013276], [-0.206424, -0.179398..."
4,brandenburg_gate;60770994_853214983-17262282_1...,"[-169.93604894380934, -1764.1683388062875, -11...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.539844, -0.200452], [-0.539009, -0.199829...","[[-0.021437, -0.182804], [-0.021437, -0.182804..."
...,...,...,...,...,...
79266,trevi_fountain;56971898_9693937883-14195208_27...,"[41.60473813297091, 57.02306644849803, 15.2297...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.483343, 0.078887], [-0.482614, 0.026122],...","[[-0.149332, 0.065841], [-0.03244, -0.155495],..."
79267,trevi_fountain;56971898_9693937883-42513253_62...,"[25.15387887183869, 2.5893603006884285, 8.1282...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.481236, 0.067917], [-0.481236, 0.067917],...","[[0.007934, 0.039083], [0.007934, 0.039083], [..."
79268,trevi_fountain;56971898_9693937883-45798320_11...,"[11.617296416650131, 2.344121118068586, 4.0608...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.482612, 0.022848], [-0.470882, 0.03304], ...","[[-0.095369, -0.295496], [-0.231158, -0.079859..."
79269,trevi_fountain;56971898_9693937883-45996779_31...,"[80.49119167083383, -373.96686954152926, 55.66...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.481219, 0.114554], [-0.475259, 0.114595],...","[[-0.027538, 0.058304], [-0.024068, -0.050028]..."


In [23]:
sub[sub.fundamental_matrix.isna()]

,sample_id,fundamental_matrix,mask,inliers1,inliers2


In [24]:
scaling_data = pd.read_csv(os.path.join(EXT_DATA_DIR, 'train/scaling_factors.csv'), index_col=0).set_index('scene')

In [25]:
thresholds_q = np.linspace(1, 10, 10)
thresholds_t = np.geomspace(0.2, 5, 10)

In [26]:
importlib.reload(wrp)

evals = wrp.evaluate_results(dataset, sub, scaling=scaling_data, thresholds_q=thresholds_q, thresholds_t=thresholds_t)

Evaluating for brandenburg_gate
Evaluating for british_museum
Evaluating for buckingham_palace
Evaluating for colosseum_exterior
Evaluating for lincoln_memorial_statue
Evaluating for notre_dame_front_facade
Evaluating for pantheon_exterior
Evaluating for sacre_coeur
Evaluating for sagrada_familia
Evaluating for taj_mahal
Evaluating for temple_nara_japan
Evaluating for trevi_fountain


In [27]:
# Extract scene names from sample_id
evals['scene'] = evals['sample_id'].apply(lambda x: x.split(';')[0])

# Group by scene and calculate mean accuracy measures
scene_accuracy = evals.groupby('scene').agg({
    'maa': 'mean'
}).reset_index()

print(scene_accuracy)

                      scene       maa
0          brandenburg_gate  0.007108
1            british_museum  0.010124
2         buckingham_palace  0.002152
3        colosseum_exterior  0.001107
4   lincoln_memorial_statue  0.015932
5   notre_dame_front_facade  0.007197
6         pantheon_exterior  0.004875
7               sacre_coeur  0.002055
8           sagrada_familia  0.011926
9                 taj_mahal  0.008083
10        temple_nara_japan  0.012360
11           trevi_fountain  0.002732


In [28]:
importlib.reload(du)

test_data = du.DatasetLoader(os.path.join(EXT_DATA_DIR), train_mode=False)


In [29]:
importlib.reload(wrp)

test_data = wrp.preprocess_data(test_data, preprocessor, extractor)

In [30]:
wrp.sample_pairs_for_run(test_data, 1000)

[+] Processing scene "notre_dame_front_facade": found 4923 pairs (will keep 1000)
[+] Processing scene "st_pauls_cathedral": found 5564 pairs (will keep 1000)
[+] Processing scene "st_peters_square": found 3403 pairs (will keep 1000)
[+] Processing scene "trevi_fountain": found 4950 pairs (will keep 1000)


In [31]:
test_data = wrp.extract_features(test_data, extractor)

Scenes: 100%|██████████| 4/4 [00:15<00:00,  3.84s/it]


In [32]:
importlib.reload(wrp)

wrp.match_features(test_data, matcher, covisibility_threshold=0.1)

Matching features for scene: notre_dame_front_facade
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene notre_dame_front_facade: 19.73 seconds
Matching features for scene: st_pauls_cathedral
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene st_pauls_cathedral: 11.46 seconds
Matching features for scene: st_peters_square
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene st_peters_square: 11.15 seconds
Matching features for scene: trevi_fountain
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene trevi_fountain: 19.69 seconds


In [33]:
test_data.scenes_data['st_pauls_cathedral'].covisibility.head()

,x,keypoints1,keypoints2,im1,im2
8355,NaN,"(12.847030, 163.223846);(15.014727, 469.048767...","(366.295776, 209.655945);(33.882420, 476.94717...",94451439_81070120,01207461_5308976270
8356,NaN,"(4.062292, 361.042084);(8.040526, 242.644150);...","(226.593643, 468.719940);(306.165894, 189.9059...",92227797_2634489539,75421800_5039453302
8358,NaN,"(3.115355, 261.354614);(12.197722, 217.873627)...","(480.639801, 224.732574);(28.023870, 296.95748...",92227797_2634489539,01207461_5308976270
8365,NaN,"(9.020869, 135.725861);(9.020869, 135.725861);...","(262.126160, 175.716110);(24.152212, 349.29657...",94451439_81070120,89944336_8704609569
8370,NaN,"(7.131830, 477.096893);(9.020869, 135.725861);...","(102.518013, 296.478516);(475.883057, 43.29823...",94451439_81070120,64682734_3599756412


In [34]:
importlib.reload(wrp)

subt = wrp.estimate_fundamental_matrix(dataset=test_data, estimator=fe)

Estimating fundamental matrix for pairs in scene: notre_dame_front_facade
Estimating fundamental matrix for pairs in scene: st_pauls_cathedral
Estimating fundamental matrix for pairs in scene: st_peters_square
Estimating fundamental matrix for pairs in scene: trevi_fountain


In [35]:
subt

,sample_id,fundamental_matrix,mask,inliers1,inliers2
0,notre_dame_front_facade;80548383_5204479115-60...,"[4.9355584881103054e-05, 0.0010906855873877087...","[[1], [1], [1], [0], [0], [1], [1], [0], [0], ...","[[2.562089, 400.581482], [7.888495, 147.036682...","[[50.984619, 432.284271], [13.537509, 136.1445..."
1,notre_dame_front_facade;90958466_3436202664-83...,"[1.7377103454662714e-06, -4.972565709483175e-0...","[[0], [0], [0], [0], [0], [0], [0], [0], [1], ...","[[27.192312, 295.37674], [55.581264, 267.31939...","[[4.973747, 385.450623], [18.058308, 349.90383..."
2,notre_dame_front_facade;50531430_8292584878-17...,"[4.352545923952589e-07, -1.0794024115851953e-0...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[50.254921, 271.894775], [67.039688, 212.3981...","[[64.627235, 335.65625], [144.70192, 257.01275..."
3,notre_dame_front_facade;59331375_2244201695-53...,"[3.745374167164552e-07, -4.05124376656611e-05,...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[36.826992, 301.137054], [37.406326, 283.9612...","[[31.161291, 325.062195], [32.101929, 302.8170..."
4,notre_dame_front_facade;68031923_487578199-371...,"[2.340665802478838e-06, 5.4608164675364493e-05...","[[0], [0], [0], [0], [1], [1], [0], [0], [1], ...","[[41.165791, 270.086884], [41.165791, 270.0868...","[[23.128672, 210.645004], [23.128672, 210.6450..."
...,...,...,...,...,...
3995,trevi_fountain;58627391_2336517069-17273683_84...,"[-1.3480942436697519e-05, -0.00039204896222706...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[70.863647, 288.116058], [109.706299, 322.478...","[[199.356003, 191.188477], [465.770935, 192.53..."
3996,trevi_fountain;58627391_2336517069-43656145_15...,"[-1.4582237840026558e-06, 1.6638497131001863e-...","[[1], [1], [0], [0], [0], [0], [0], [0], [0], ...","[[8.185253, 487.308777], [9.446077, 379.07962]...","[[60.99913, 423.680542], [297.067383, 377.2177..."
3997,trevi_fountain;80288369_2336500045-12228953_78...,"[3.7284185514997564e-06, 4.949550500423393e-06...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[27.535656, 124.642822], [30.865631, 126.2289...","[[135.095245, 374.947021], [135.095245, 374.94..."
3998,trevi_fountain;65528665_8351228484-12228953_78...,"[4.7229561022674074e-07, 1.5172444263677276e-0...","[[0], [0], [0], [0], [0], [1], [0], [0], [0], ...","[[34.863174, 288.543884], [122.76992, 225.9916...","[[330.094177, 497.302521], [355.650543, 329.42..."


In [36]:
missing = test_data.test_samples[test_data.test_samples.for_exp.isna()].sample_id
missing_df = pd.DataFrame(missing, columns=['sample_id'])


In [37]:
missing_df['fundamental_matrix'] = missing_df.apply(lambda row: np.random.rand(9), axis=1)

In [38]:
final = pd.concat([subt, missing_df]).reset_index(drop=True)[['sample_id', 'fundamental_matrix']]

In [39]:
final

,sample_id,fundamental_matrix
0,notre_dame_front_facade;80548383_5204479115-60...,"[4.9355584881103054e-05, 0.0010906855873877087..."
1,notre_dame_front_facade;90958466_3436202664-83...,"[1.7377103454662714e-06, -4.972565709483175e-0..."
2,notre_dame_front_facade;50531430_8292584878-17...,"[4.352545923952589e-07, -1.0794024115851953e-0..."
3,notre_dame_front_facade;59331375_2244201695-53...,"[3.745374167164552e-07, -4.05124376656611e-05,..."
4,notre_dame_front_facade;68031923_487578199-371...,"[2.340665802478838e-06, 5.4608164675364493e-05..."
...,...,...
18835,notre_dame_front_facade;97270767_81833198-8637...,"[0.46318538283346267, 0.028781216777905017, 0...."
18836,notre_dame_front_facade;97270767_81833198-8701...,"[0.631284639876883, 0.8100532649244457, 0.2372..."
18837,notre_dame_front_facade;97270767_81833198-8779...,"[0.3927997180856452, 0.42721431224903306, 0.25..."
18838,notre_dame_front_facade;97270767_81833198-8842...,"[0.710177589387444, 0.7106038060135481, 0.8713..."


In [40]:
final[final.fundamental_matrix.isna()]

,sample_id,fundamental_matrix


In [41]:
final

,sample_id,fundamental_matrix
0,notre_dame_front_facade;80548383_5204479115-60...,"[4.9355584881103054e-05, 0.0010906855873877087..."
1,notre_dame_front_facade;90958466_3436202664-83...,"[1.7377103454662714e-06, -4.972565709483175e-0..."
2,notre_dame_front_facade;50531430_8292584878-17...,"[4.352545923952589e-07, -1.0794024115851953e-0..."
3,notre_dame_front_facade;59331375_2244201695-53...,"[3.745374167164552e-07, -4.05124376656611e-05,..."
4,notre_dame_front_facade;68031923_487578199-371...,"[2.340665802478838e-06, 5.4608164675364493e-05..."
...,...,...
18835,notre_dame_front_facade;97270767_81833198-8637...,"[0.46318538283346267, 0.028781216777905017, 0...."
18836,notre_dame_front_facade;97270767_81833198-8701...,"[0.631284639876883, 0.8100532649244457, 0.2372..."
18837,notre_dame_front_facade;97270767_81833198-8779...,"[0.3927997180856452, 0.42721431224903306, 0.25..."
18838,notre_dame_front_facade;97270767_81833198-8842...,"[0.710177589387444, 0.7106038060135481, 0.8713..."


In [42]:
formatted_final['fundamental_matrix'] = formatted_final['fundamental_matrix'].apply(lambda x: ' '.join(f'{num:.5e}' for num in x))
final.to_csv('preds.csv', index=False)

NameError: name 'formatted_final' is not defined